In [1]:
import requests
import pandas as pd

In [2]:
class CrawlerIngresse:
    ticket_office = 'ingresse'
    base_api = 'https://event-search.ingresse.com/1?state={state}&from=now&orderBy=sessions.dateTime&size=200&category={category}'
    base_url = 'https://www.ingresse.com/'
    category = [
        'festas-e-baladas',
        'shows-e-festivais'
    ]

    def get_events(self, state: str):
        all_categories_data = [d for category in self.category for d in self._get_data(state, category)['data']['hits']]
        return self._process_data(all_categories_data, state)

    def save_events_in_db(self):
        pass

    def _get_data(self, state: str, category: str):
        return requests.get(self.base_api.format(state=state, category=category)).json()
    
    def _process_data(self, raw_data: list[dict], state: str):
        return (
            pd.DataFrame({
                "name": [d['_source']['title'] for d in raw_data],
                "location_city": [d['_source']['place']['city'] for d in raw_data],
                "location_lat": [d['_source']['place']['location']['lat'] for d in raw_data],
                "location_lon": [d['_source']['place']['location']['lon'] for d in raw_data],
                "location_name": [d['_source']['place']['name'] for d in raw_data],
                "location_street": [d['_source']['place']['street'] for d in raw_data],
                "location_zipcode": [d['_source']['place']['zip'] for d in raw_data],
                "date": [[x['dateTime'] for x in d['_source']['sessions'] if x['status'] != 'finished'] for d in raw_data],
                "updated_at": [d['_source']['updatedAt'] for d in raw_data],
                "categories": [[x['slug'] for x in d['_source']['categories']] for d in raw_data],
                "poster_medium": [d['_source']['poster']['medium'] for d in raw_data],
                "poster_small": [d['_source']['poster']['small'] for d in raw_data],
                "external_ticket_id": [d['_source']['id'] for d in raw_data],
                "ticket_url": [self.base_url + d['_source']['slug'] for d in raw_data],
            })
            .explode('date')
            .reset_index(drop=True)
            .assign(location=state)
        )

        

In [3]:
crawler = CrawlerIngresse()
df = crawler.get_events('rj')

In [4]:
df

,name,location_city,location_lat,location_lon,location_name,location_street,location_zipcode,date,updated_at,categories,poster_medium,poster_small,external_ticket_id,ticket_url,location
0,SAMBA INDEPENDENTE DOS BONS COSTUMES,Rio de Janeiro,-22.912416,-43.180633,Fundição Progresso,"Rua dos Arcos, 24",20230-060,2023-07-13T21:00:00+00:00,2023-07-03T17:21:50.922918+00:00,[festas-e-baladas],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,65113,https://www.ingresse.com/samba-independente-do...,rj
1,SAMBA INDEPENDENTE DOS BONS COSTUMES,Rio de Janeiro,-22.912416,-43.180633,Fundição Progresso,"Rua dos Arcos, 24",20230-060,2023-07-20T21:00:00+00:00,2023-07-03T17:21:50.922918+00:00,[festas-e-baladas],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,65113,https://www.ingresse.com/samba-independente-do...,rj
2,SAMBA INDEPENDENTE DOS BONS COSTUMES,Rio de Janeiro,-22.912416,-43.180633,Fundição Progresso,"Rua dos Arcos, 24",20230-060,2023-07-27T21:00:00+00:00,2023-07-03T17:21:50.922918+00:00,[festas-e-baladas],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,65113,https://www.ingresse.com/samba-independente-do...,rj
3,PARQUE BAR - Hip Hop Exp,Rio de Janeiro,-22.973693,-43.216394,Parque Bar,"Avenida Borges de Medeiros, 1518",22470-003,2023-07-13T20:00:00+00:00,2023-07-11T20:47:55.946627+00:00,[festas-e-baladas],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,65251,https://www.ingresse.com/parque-barr-hip-hop-exp,rj
4,Sweet Hills,Rio de Janeiro,-22.955727,-43.166980,Hills,"Praça General Tibúrcio, 520",22290-270,2023-07-13T22:00:00+00:00,2023-07-07T23:27:29.337488+00:00,"[festas-e-baladas, eletronica, funk, premium]",https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,65203,https://www.ingresse.com/sweet-hills,rj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Camarote Taylor Swift - Ingresso Show - Rio de...,Rio de Janeiro,-22.893275,-43.292313,Estádio Olímpico João Havelange,"Rua José dos Reis, 425",20770-001,2023-11-17T21:00:00+00:00,2023-06-27T19:29:08.417715+00:00,[shows-e-festivais],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,64618,https://www.ingresse.com/taylor-swift-rj-ingre...,rj
113,Transporte - Taylor Swift | The Eras - Rio De ...,Rio de Janeiro,-22.893275,-43.292313,Estádio Olímpico Nilton Santos,"Rua José dos Reis, 425",20770-001,2023-11-18T23:59:00+00:00,2023-07-07T22:24:40.829907+00:00,[shows-e-festivais],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,64535,https://www.ingresse.com/transporte-taylor-swi...,rj
114,Transporte - Taylor Swift | The Eras - Rio De ...,Rio de Janeiro,-22.893275,-43.292313,Estádio Olímpico Nilton Santos,"Rua José dos Reis, 425",20770-001,2023-11-19T23:59:00+00:00,2023-07-07T22:24:40.829907+00:00,[shows-e-festivais],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,64535,https://www.ingresse.com/transporte-taylor-swi...,rj
115,Transporte - Taylor Swift | The Eras - Rio De ...,Rio de Janeiro,-22.893275,-43.292313,Estádio Olímpico Nilton Santos,"Rua José dos Reis, 425",20770-001,2023-11-17T23:59:00+00:00,2023-07-07T22:24:40.829907+00:00,[shows-e-festivais],https://event-kraken.s3.amazonaws.com/event/po...,https://event-kraken.s3.amazonaws.com/event/po...,64535,https://www.ingresse.com/transporte-taylor-swi...,rj
